In [ ]:
import os
from pathlib import Path
from mne.io import read_raw, read_raw_ant
from mne_bids import BIDSPath, write_raw_bids
from tqdm.contrib import tzip
from tqdm import tqdm
from mne import set_log_level
set_log_level("Error")

In [ ]:
root_dir = Path("/Volumes/Extreme_SSD/payam_data/Tide_data")
bids_dir = root_dir / "BIDS"

site = "regensburg"
data_dir = root_dir / site

files = [Path("/Users/payamsadeghishabestari/Downloads/OneDrive_1_13-11-2025/50044_restingstate.vhdr"),
        Path("/Users/payamsadeghishabestari/Downloads/OneDrive_2_13-11-2025/50033_restingstate.vhdr"),
        ]
        

subject_ids = [f.stem[:5] for f in files]
for fname, subject_id in tzip(files, subject_ids,
                                        total=len(subject_ids),
                                        desc=f"Converting {site}"):
    print(fname)
    print(subject_ids)
    raw = read_raw(fname)
    bids_path = BIDSPath(
                subject=subject_id,
                session="01",
                task="rest",
                datatype="eeg",
                description=site,
                root=bids_dir
            )
    write_raw_bids(raw, bids_path=bids_path, overwrite=True)

extract behavioural metrics

In [77]:
from pathlib import Path
from warnings import warn
import pandas as pd
behavioural_dir = Path.cwd().parent / "material" 

site_map = {
        "1": "austin",
        "2": "dublin",
        "3": "ghent",
        "4": "illinois",
        "5": "regensburg",
        "6": "tuebingen",
        "7": "zuerich"
        }

quest_cols = [
            "study_id",
            "intro_gender",
            "esit_a1", # age
            "hq_score",
            "hq_attentional_score",
            "hq_social_score",
            "hq_emotional_score",
            "hads_d_score",
            "hads_a_score",
            "esit_a8_tin", # group
            "esit_a8_hl", # HL
            "tfi_score",
            "thi_score",
            "psq_score" # stress level
            ]

audio_cols = [
            "SubjectID",
            "PTA",          
            "HF_PTA",       
        ]


dfs = []
for site in site_map.values():
    site_code = site.upper()[:3]
    quest_fname = behavioural_dir / "questionnaires" / f"Questionnaire_data_TIDE_{site_code}.csv"
    audio_fname = behavioural_dir / "audiograms" / f"Audiometry_data_TIDE_{site_code}.csv"
    
    if not quest_fname.is_file() or not audio_fname.is_file():
        warn(f"Missing data for site '{site}': "
            f"{'questionnaire' if not quest_fname.is_file() else ''} "
            f"{'audiometry' if not audio_fname.is_file() else ''}. Skipping this site.")
        continue
    
    df_quest = pd.read_csv(quest_fname, sep=None, engine="python")
    df_quest["site"] = site

    missing = set(quest_cols) - set(df_quest.columns)
    if missing:
        warn(f"{site}: missing columns {missing}, skipping.")
        continue

    dfs.append(df_quest[quest_cols + ["site"]])

df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all[["site", *quest_cols]]



/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1608/4071921970.py:47: UserWarning: Missing data for site 'dublin': questionnaire . Skipping this site.
  warn(f"Missing data for site '{site}': "
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1608/4071921970.py:47: UserWarning: Missing data for site 'ghent': questionnaire . Skipping this site.
  warn(f"Missing data for site '{site}': "
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1608/4071921970.py:57: UserWarning: illinois: missing columns {'intro_gender', 'study_id'}, skipping.
  warn(f"{site}: missing columns {missing}, skipping.")
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1608/4071921970.py:47: UserWarning: Missing data for site 'zuerich': questionnaire audiometry. Skipping this site.
  warn(f"Missing data for site '{site}': "


In [79]:
df_all

,site,study_id,intro_gender,esit_a1,hq_score,hq_attentional_score,hq_social_score,hq_emotional_score,hads_d_score,hads_a_score,esit_a8_tin,esit_a8_hl,tfi_score,thi_score,psq_score
0,austin,10003,1,69,5,2,2,1,0,0,0.0,1.0,13.2,12.0,0.277778
1,austin,10011,2,21,6,1,1,4,1,3,0.0,0.0,NaN,NaN,0.288889
2,austin,10004,1,20,11,1,8,2,5,7,0.0,0.0,32.0,36.0,0.500000
3,austin,10010,1,56,1,0,1,0,1,5,0.0,1.0,23.2,18.0,0.355556
4,austin,10013,2,28,5,2,2,1,0,14,1.0,1.0,NaN,NaN,0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,tuebingen,60088,2,20,5,2,3,0,12,9,0.0,0.0,NaN,NaN,0.411111
240,tuebingen,60090,2,24,8,4,1,3,0,7,0.0,0.0,12.8,10.0,0.211111
241,tuebingen,60092,2,55,8,2,2,4,0,1,0.0,0.0,NaN,NaN,0.300000
242,tuebingen,60093,2,51,2,0,1,1,0,1,0.0,0.0,NaN,NaN,0.322222


In [ ]:
"illinois: missing columns {'intro_gender', 'study_id'}"

,esit_a1,intro_gender,esit_a8_tin,esit_a8_hl
count,244.000000,244.000000,242.000000,242.000000
mean,42.881148,1.536885,0.252066,0.247934
std,17.054929,0.499663,0.435099,0.432708
min,18.000000,1.000000,0.000000,0.000000
25%,26.000000,1.000000,0.000000,0.000000
50%,44.000000,2.000000,0.000000,0.000000
75%,58.000000,2.000000,0.750000,0.000000
max,70.000000,2.000000,1.000000,1.000000


In [ ]:
## for regensburg
## 	PTA	THI	TFI	LATERALITY	SUBTYPE	PSQ

## check email